# Derivados Financieros 2023 - Clase 3



Plan de la clase:

    1) Modelo Binomial
    2) Modificacion para ejercicio temprano (Americanas)
    3) Jugando con el pricer
    4) Black Scholes
    5) Mercado vs Modelo

## 1) Modelo Binomial

In [ ]:
import math
import numpy as np

import sys
sys.path.append('..')

from Codigo.opcion_europea_bin import opcion_europea_bin
from Codigo.opcion_europea_bin_c import opcion_europea_bin_c
from Codigo.opcion_americana_bin import opcion_americana_bin
from Codigo.opcion_europea_bs import opcion_europea_bs



Definicion:
    Calculador del precio de una opcion Europea con el modelo del Arbol Binomial (CRR)

Inputs:
    - tipo : string - Tipo de contrato entre ["CALL","PUT"]
    - S : float - Spot price del activo
    - K : float - Strike price del contrato
    - T : float - Tiempo hasta la expiracion (en años)
    - r : float - Tasa 'libre de riesgo' (anualizada)
    - sigma : float - Volatilidad implicita (anualizada)
    - div : float - Tasa de dividendos continuos (anualizada)
    - pasos : int - Cantidad de pasos del arbol binomial

Outputs:
    - precio_BIN: float - Precio del contrato


In [ ]:
#Paramtros del metodo

tipo = "C"
S = 100
K = 100
T = 1
r = 0.05
sigma = 0.25
div = 0
pasos = 20


precio = opcion_europea_bin(tipo, S, K, T, r, sigma, div, pasos)
precio

## Desglosando el metodo

### 1) Inicializo los parametros

In [ ]:
tipo = "C"
S = 100
K = 100
T = 1
r = 0.05
sigma = 0.25
div = 0
pasos = 3

### 2) Variables auxiliares

In [ ]:
#auxiliares
dt = T / pasos
tasa_forward = math.exp((r - div) * dt)
descuento = math.exp(-r * dt)


### 3) Parametros del Modelo (en este caso el CRR)

In [ ]:
#modelo CRR
u = math.exp(sigma * math.pow(dt, 0.5))
d = 1 / u

#probabilidad de riesgo neutral
q_prob = (tasa_forward - d) / (u - d)

q_prob


### 4) Vector de precios finales (ultima rama del arbol)

In [ ]:
#Precios finales
ST_precios=np.zeros((pasos+1))

for i in range(0,pasos+1):
    ST_precios[pasos-i] = math.pow(u, 2 * i - pasos) * S
    
ST_precios

### 5) Matriz (representa el arbol) de precios de la opcion -  la ultima columna representa el Payoff

In [ ]:
from pprint import pprint as pp
#Matriz de precios de la opcion
opcion_precios = np.zeros((pasos+1, pasos+1))

opcion_precios

In [ ]:
#Payoff
for i in range (0, pasos+1):
    if tipo == "P":
        opcion_precios[i][pasos] = max(0, (K - ST_precios[i]))
    elif tipo == "C":
        opcion_precios[i][pasos] = max(0, (ST_precios[i] - K))
        
pp(opcion_precios)

### 6) Recorro el arbol columna por columna

In [ ]:
j = 1
for i in range(0, pasos+1 - j):
    opcion_precios[i][pasos - j] = descuento * (q_prob * opcion_precios[i][pasos - j + 1] + (1  - q_prob) * opcion_precios[i + 1][pasos - j + 1])

opcion_precios   

In [ ]:
j = 2
for i in range(0, pasos+1 - j):
    opcion_precios[i][pasos - j] = descuento * (q_prob * opcion_precios[i][pasos - j + 1] + (1  - q_prob) * opcion_precios[i + 1][pasos - j + 1])

opcion_precios 

In [ ]:
j = 3
for i in range(0, pasos+1 - j):
    opcion_precios[i][pasos - j] = descuento * (q_prob * opcion_precios[i][pasos - j + 1] + (1  - q_prob) * opcion_precios[i + 1][pasos - j + 1])

opcion_precios 

### 7) Precio de la opcion, primer nodo del arbol 

In [ ]:
precio_BIN_eur = opcion_precios[0][0]
pp(precio_BIN_eur)

## Formula Cerrada

Volvemos a inicializar todas las variables


In [ ]:
import operator as op
from functools import reduce
def ncr(n, r):
    r = min(r, n-r)
    numer = reduce(op.mul, range(n, n-r, -1), 1)
    denom = reduce(op.mul, range(1, r+1), 1)
    return numer // denom  # or / in Python 2


def nCr(n, r):
    f = math.factorial
    return f(n) / f(r) / f(n - r)

In [ ]:
tipo = "C"
S = 100
K = 100
T = 1
r = 0.05
sigma = 0.25
div = 0
pasos = 1000


#auxiliares
dt = T / pasos
tasa_forward = math.exp((r - div) * dt)
descuento = math.exp(-r * dt)

#modelo CRR
u = math.exp(sigma * math.pow(dt, 0.5))
d = 1 / u
#probabilidad de riesgo neutral
q_prob = (tasa_forward - d) / (u - d)


In [ ]:
temp = 0

for k in range(pasos):
    if tipo == "C":
        payoff = max(0, S * math.pow(u,k) * math.pow(d,pasos-k)-K)
    elif tipo == "P":
        payoff = max(0, K - S * math.pow(u, k) * math.pow(d, pasos - k))
    temp = temp +ncr(pasos, k) * math.pow(q_prob,k) * math.pow((1-q_prob),pasos-k) * payoff

precio_BIN_c = math.exp(-r*T)*temp
    
precio_BIN_c

## 2) Americana

In [ ]:
tipo = "C"
S = 100
K = 100
T = 1
r = 0.05
sigma = 0.25
div = 0
pasos = 1000


#auxiliares
dt = T / pasos
tasa_forward = math.exp((r - div) * dt)
descuento = math.exp(-r * dt)
#modelo CRR
u = math.exp(sigma * math.pow(dt, 0.5))
d = 1 / u
#probabilidad de riesgo neutral
q_prob = (tasa_forward - d) / (u - d)


#Precios finales
ST_precios=np.zeros((pasos+1))

for i in range(0,pasos+1):
    ST_precios[pasos-i] = math.pow(u, 2 * i - pasos) * S
    
ST_precios
opcion_precios = np.zeros((pasos+1, pasos+1))




En el caso de una opcion americana, en cada punto del nodo tendremos que evaluar si nos conviene ejercer o no. Eso se resuelve de manera muy simple.

In [ ]:
for j in range(1, pasos+1):
    for i in range(0, pasos+1 - j):
        # Calculo primero el valor como si no hubiera ejercicio temprano (europeo)
        eur = q_prob * opcion_precios[i][pasos - j + 1] + (1  - q_prob) * opcion_precios[i + 1][pasos - j + 1]
        #dependiendo del contrato, comparo el valor del valor 'europeo' contra ejercer en ese momento.
        # S * math.pow(u,-2*i+pasos-j) codifica al arbol de los S
        if tipo == "P":
            opcion_precios[i][pasos - j] = descuento * max(eur, K - S * math.pow(u,-2*i+pasos-j))
        elif tipo == "C":
            opcion_precios[i][pasos - j] = descuento * max(eur, S * math.pow(u,-2*i+pasos-j) - K)

precio_BIN_ame = opcion_precios[0][0]
pp(precio_BIN_ame) 



# 3) Jugando con el pricer

In [ ]:
import matplotlib.pyplot as plt

## Precio y payoff con respecto a S

In [ ]:
tipo = "C"
S = 100
K = 100
T = 1
r = 0.05
sigma= 0.25
div =0
pasos = 1000

S_MAX = 2*S
S_vec = np.linspace(0,S_MAX,50)      
precios = np.zeros(len(S_vec))
payoff = np.zeros(len(S_vec))



S_vec 


In [ ]:
for index in range(len(S_vec)):
    precios[index] = opcion_europea_bin(tipo, S_vec[index], K, T, r, sigma, div, pasos)
    payoff[index] = max(0, S_vec[index]-K)
plt.plot(S_vec,precios,'r', label='Precio')
plt.plot(S_vec,payoff,'g', label='Payoff')
plt.axvline(x=K, color = 'black', linestyle='--')

plt.legend()

plt.xlabel('Spot')
plt.ylabel('Precio del Call')
plt.title('Precio/Payoff de una opcion Call a tiempo inicial con respecto al spot')

plt.show()

In [ ]:
tipo = "P"
S = 100
K = 100
T = 1
r = 0.05
sigma= 0.25
div =0
pasos = 1000

S_MAX = 2*S
S_vec = np.linspace(0,S_MAX,50)      
precios = np.zeros(len(S_vec))
payoff = np.zeros(len(S_vec))




for index in range(len(S_vec)):
    precios[index] = opcion_europea_bin_c(tipo, S_vec[index], K, T, r, sigma, div, pasos)
    payoff[index] = max(0, K-S_vec[index])
plt.plot(S_vec,precios,'r', label='Precio')
plt.plot(S_vec,payoff,'g', label='Payoff')
plt.axvline(x=K, color = 'black', linestyle='--')

plt.legend()

plt.xlabel('Spot')
plt.ylabel('Precio del Put')
plt.title('Precio/Payoff de una opcion Put a tiempo inicial con respecto al spot')

plt.show()

In [ ]:
S_vec

## Precio con respecto al Strike

In [ ]:
tipo = "C"
S = 100
K = 100
T = 1
r = 0.05
sigma= 0.25
div =0
pasos = 1000

K_MAX = 2*S
K_vec = np.linspace(0,K_MAX,50)      
precios = np.zeros(len(K_vec))




for index in range(len(K_vec)):
    precios[index] = opcion_europea_bin_c(tipo, S, K_vec[index], T, r, sigma, div, pasos)

plt.plot(K_vec,precios,'r', label='Strike')
plt.axvline(x=S, color = 'black', linestyle='--')

plt.legend()

plt.xlabel('Strike')
plt.ylabel('Precio del Call')
plt.title('Precio de una opcion Call a tiempo inicial con respecto al strike')

plt.show()

In [ ]:
tipo = "P"
S = 100
K = 100
T = 1
r = 0.05
sigma= 0.25
div =0
pasos = 1000

K_MAX = 2*S
K_vec = np.linspace(0,K_MAX,50)      
precios = np.zeros(len(K_vec))
payoff = np.zeros(len(K_vec))




for index in range(len(K_vec)):
    precios[index] = opcion_europea_bin_c(tipo, S, K_vec[index], T, r, sigma, div, pasos)

plt.plot(K_vec,precios,'r', label='Strike')
plt.axvline(x=S, color = 'black', linestyle='--')

plt.legend()

plt.xlabel('Spot')
plt.ylabel('Precio del Put')
plt.title('Precio de una opcion Put a tiempo inicial con respecto al strike')

plt.show()

## Precio con respecto a sigma

In [ ]:
tipo = "C"
S = 100
K = 100
T = 1
r = 0.05
sigma= 0.25
div =0
pasos = 1000

sigma_MAX = 8*sigma
sigma_vec = np.linspace(0,sigma_MAX,50)      
precios = np.zeros(len(sigma_vec))

for index in range(1,len(sigma_vec)):
    precios[index] = opcion_europea_bin_c(tipo, S, K, T, r, sigma_vec[index], div, pasos)
plt.plot(sigma_vec,precios,'r', label='Precio')
plt.legend()

plt.xlabel('Volatilidad implicita')
plt.ylabel('Precio del Call')
plt.title('Precio de una opcion Call a tiempo inicial con respecto a la volatilidad implicita')

plt.show()

In [ ]:
tipo = "P"
S = 100
K = 100
T = 1
r = 0.05
sigma= 0.25
div =0
pasos = 1000

sigma_MAX = 8*sigma
sigma_vec = np.linspace(0,sigma_MAX,50)      
precios = np.zeros(len(sigma_vec))

for index in range(1,len(sigma_vec)):
    precios[index] = opcion_europea_bin_c(tipo, S, K, T, r, sigma_vec[index], div, pasos)
plt.plot(sigma_vec,precios,'r', label='Precio')
plt.legend()

plt.xlabel('Volatilidad implicita')
plt.ylabel('Precio del Put')
plt.title('Precio de una opcion Put a tiempo inicial con respecto a la volatilidad implicita')

plt.show()

## Precio con respecto a TTM

In [ ]:
tipo = "C"
S = 100
K = 100
T = 1
r = 0.05
sigma= 0.25
div =0.5
pasos = 1000

T_MAX = 2*T
T_vec = np.linspace(0,T_MAX,50)      
precios = np.zeros(len(T_vec))

for index in range(1,len(T_vec)):
    precios[index] = opcion_europea_bin_c(tipo, S, K, T_vec[index], r, sigma, div, pasos)
plt.plot(T_vec,precios,'r', label='Precio')
plt.legend()

plt.xlabel('Tiempo de expiracion en años')
plt.ylabel('Precio del Call')
plt.title('Precio de una opcion Call a tiempo inicial con respecto al tiempo de expiracion')

plt.show()

## Convergencia del Arbol Binomial

In [ ]:
tipo = "C"
S = 100
K = 100
T = 1
r = 0.05
sigma= 0.25
div =0

pasos_vec = [1,2,3,4,5,6,7,8,9,10,12,14,16,18,20,25,30,40,50,75,100,125,150,200,250,300]      
precios = np.zeros(len(pasos_vec))

for index in range(len(pasos_vec)):
    precios[index] = opcion_europea_bin(tipo, S, K, T, r, sigma, div, pasos_vec[index])
    

In [ ]:
plt.plot(pasos_vec,precios,'*', label='Precio')
plt.legend()

plt.xlabel('Pasos del arbol Binomial')
plt.ylabel('Precio del Call')
plt.title('Precio de una opcion Call a tiempo inicial con respecto al numero de pasos del arbol')

plt.show()

## Europeas - Americanas


### CALL sin dividendos

In [ ]:
tipo = "C"
K = 100
T = 1
r = 0.05
sigma= 0.25
div =0
pasos = 200

S_vec = np.linspace(0.5*S,1.5*S,50)      
precios_eur = np.zeros(len(S_vec))
precios_ame = np.zeros(len(S_vec))

payoff = np.zeros(len(S_vec))


for index in range(len(S_vec)):
    precios_eur[index] = opcion_europea_bin(tipo, S_vec[index], K, T, r, sigma, div, pasos)
    precios_ame[index] = opcion_americana_bin(tipo, S_vec[index], K, T, r, sigma, div, pasos)
    
    payoff[index] = max(0, S_vec[index]-K)

plt.plot(S_vec,precios_eur,'r', label='Precio Europea')
plt.plot(S_vec,precios_ame,'b', label='Precio Americana')

plt.plot(S_vec,payoff,'g', label='Payoff')
plt.axvline(x=K, color = 'black', linestyle='--')

plt.legend()

plt.xlabel('Spot')
plt.ylabel('Precio del Call')
plt.title('Precio/Payoff de una opcion Call a tiempo inicial con respecto al spot')

plt.show()


print("Precio Opcion Europea: ", opcion_europea_bin(tipo, S, K, T, r, sigma, div, pasos))
print("Precio Opcion Americana: ", opcion_americana_bin(tipo, S, K, T, r, sigma, div, pasos))


### PUT sin dividendos

In [ ]:
tipo = "P"

precios_eur = np.zeros(len(S_vec))
precios_ame = np.zeros(len(S_vec))

payoff = np.zeros(len(S_vec))

for index in range(len(S_vec)):
    precios_eur[index] = opcion_europea_bin(tipo, S_vec[index], K, T, r, sigma, div, pasos)
    precios_ame[index] = opcion_americana_bin(tipo, S_vec[index], K, T, r, sigma, div, pasos)
    
    payoff[index] = max(0, K -S_vec[index])

plt.plot(S_vec,precios_eur,'r', label='Precio Europea')
plt.plot(S_vec,precios_ame,'b', label='Precio Americana')

plt.plot(S_vec,payoff,'g', label='Payoff')
plt.axvline(x=K, color = 'black', linestyle='--')

plt.legend()

plt.xlabel('Spot')
plt.ylabel('Precio del Put')
plt.title('Precio/Payoff de una opcion Put a tiempo inicial con respecto al spot')

plt.show()


print("Precio Opcion Europea: ", opcion_europea_bin(tipo, S, K, T, r, sigma, div, pasos))
print("Precio Opcion Americana: ", opcion_americana_bin(tipo, S, K, T, r, sigma, div, pasos))


### CALL con dividendos

In [ ]:
tipo = "C"

div = 0.1

precios_eur = np.zeros(len(S_vec))
precios_ame = np.zeros(len(S_vec))

payoff = np.zeros(len(S_vec))

for index in range(len(S_vec)):
    precios_eur[index] = opcion_europea_bin(tipo, S_vec[index], K, T, r, sigma, div, pasos)
    precios_ame[index] = opcion_americana_bin(tipo, S_vec[index], K, T, r, sigma, div, pasos)
    
    payoff[index] = max(0, S_vec[index]-K)

plt.plot(S_vec,precios_eur,'r', label='Precio Europea')
plt.plot(S_vec,precios_ame,'b', label='Precio Americana')

plt.plot(S_vec,payoff,'g', label='Payoff')
plt.axvline(x=K, color = 'black', linestyle='--')

plt.legend()

plt.xlabel('Spot')
plt.ylabel('Precio del Call')
plt.title('Precio/Payoff de una opcion Call a tiempo inicial con respecto al spot')

plt.show()


print("Precio Opcion Europea: ", opcion_europea_bin(tipo, S, K, T, r, sigma, div, pasos))
print("Precio Opcion Americana: ", opcion_americana_bin(tipo, S, K, T, r, sigma, div, pasos))

### PUT con dividendos

In [ ]:
tipo = "P"

div = 0.1

precios_eur = np.zeros(len(S_vec))
precios_ame = np.zeros(len(S_vec))

payoff = np.zeros(len(S_vec))

for index in range(len(S_vec)):
    precios_eur[index] = opcion_europea_bin(tipo, S_vec[index], K, T, r, sigma, div, pasos)
    precios_ame[index] = opcion_americana_bin(tipo, S_vec[index], K, T, r, sigma, div, pasos)
    
    payoff[index] = max(0, K -S_vec[index])

plt.plot(S_vec,precios_eur,'r', label='Precio Europea')
plt.plot(S_vec,precios_ame,'b', label='Precio Americana')

plt.plot(S_vec,payoff,'g', label='Payoff')
plt.axvline(x=K, color = 'black', linestyle='--')

plt.legend()

plt.xlabel('Spot')
plt.ylabel('Precio del Put')
plt.title('Precio/Payoff de una opcion Put a tiempo inicial con respecto al spot')

plt.show()


print("Precio Opcion Europea: ", opcion_europea_bin(tipo, S, K, T, r, sigma, div, pasos))
print("Precio Opcion Americana: ", opcion_americana_bin(tipo, S, K, T, r, sigma, div, pasos))

In [ ]:
div = 0
tipo = "P"
r = 0.05

precios_eur = np.zeros(len(T_vec))
precios_ame = np.zeros(len(T_vec))



for index in range(1,len(T_vec)):
    precios_eur[index] = opcion_europea_bin(tipo, S, K, T_vec[index], r, sigma, div, pasos)
    precios_ame[index] = opcion_americana_bin(tipo, S, K, T_vec[index], r, sigma, div, pasos)
    
    payoff[index] = max(0, S_vec[index]-K)

plt.plot(T_vec,precios_eur,'r', label='Precio Europea')
plt.plot(T_vec,precios_ame,'b', label='Precio Americana')

plt.legend()

plt.xlabel('TTM')
plt.ylabel('Precio del Put')
plt.title('Precio de una opcion Put a tiempo inicial con respecto al TTM')

plt.show()


print("Precio Opcion Europea: ", opcion_europea_bin(tipo, S, K, T, r, sigma, div, pasos))
print("Precio Opcion Americana: ", opcion_americana_bin(tipo, S, K, T, r, sigma, div, pasos))

## 4) Black Scholes

Si llegamos hasta aca, entonces estamos en condiciones de ver la formula de Black Scholes. Veremos el modelo en la Clase 4

In [ ]:
tipo = "C"
K = 100
T = 1
r = 0.05
sigma= 0.25
div =0

from scipy.stats import norm


In [ ]:


#Defino los ds
d1 = (math.log(S / K) + (r - div + 0.5 * sigma * sigma) * T) / sigma / math.sqrt(T)
d2 = (math.log(S / K) + (r - div - 0.5 * sigma * sigma) * T) / sigma / math.sqrt(T)

if tipo == "C":
    precio_BS = math.exp(-div*T) *S * norm.cdf(d1) - K * math.exp(-r * T) * norm.cdf(d2)
elif tipo == "P":
    precio_BS = K * math.exp(-r * T) * norm.cdf(-d2) - math.exp(-div*T) * S * norm.cdf(-d1)
    
precio_BS    

## Convergencia del Binomial hacia el precio de Black Scholes

In [ ]:
tipo = "P"
S = 100
K = 100
T = 1
r = 0.05
sigma= 0.25
div =0.2

pasos_vec = [1,2,3,4,5,6,7,8,9,10,12,14,16,18,20,25,30,40,50,75,100,125,150,200,250,300]      
precios = np.zeros(len(pasos_vec))

for index in range(len(pasos_vec)):
    precios[index] = opcion_europea_bin(tipo, S, K, T, r, sigma, div, pasos_vec[index])
 
precio_BS = opcion_europea_bs(tipo, S, K, T, r, sigma, div)



In [ ]:
plt.axhline(y=precio_BS, color = 'black', linestyle='--', label='Precio BlackSholes')
plt.plot(pasos_vec,precios,'*', label='Precio Binomial')

plt.legend()

plt.xlabel('Pasos del arbol Binomial')
plt.ylabel('Precio de la opcion')
plt.title('Precio de una opcion a tiempo inicial con respecto al numero de pasos del arbol')

plt.show()

## Convergencia del Binomial (con Americana) hacia el precio de Black Scholes

In [ ]:
tipo = "P"
S = 100
K = 100
T = 1
r = 0.05
sigma= 0.25
div =0.

pasos_vec = [1,2,3,4,5,6,7,8,9,10,12,14,16,18,20,25,30,40,50,75,100,125,150,200,250,300]      
precios_eur = np.zeros(len(pasos_vec))
precios_ame = np.zeros(len(pasos_vec))


for index in range(len(pasos_vec)):
    precios_eur[index] = opcion_europea_bin(tipo, S, K, T, r, sigma, div, pasos_vec[index])
    precios_ame[index] = opcion_americana_bin(tipo, S, K, T, r, sigma, div, pasos_vec[index])
 
precio_BS = opcion_europea_bs(tipo, S, K, T, r, sigma, div)



In [ ]:
plt.axhline(y=precio_BS, color = 'black', linestyle='--', label='Precio BlackSholes')
plt.plot(pasos_vec,precios_eur,'*', label='Precio Binomial Europea')
plt.plot(pasos_vec,precios_ame,'.', label='Precio Binomial Americana')

plt.legend()

plt.xlabel('Pasos del arbol Binomial')
plt.ylabel('Precio de la opcion')
plt.title('Precio de una opcion Call a tiempo inicial con respecto al numero de pasos del arbol')

plt.show()

## 5) Mercado VS Modelo
## a) Saco una opcion del mercado y calculo su precio - NYSE

In [ ]:
# Tomo un ticker e invoco al codigo de la clase pasada

from Codigo.data_nyse import *
ticker = 'TSLA'
data = yf.Ticker(ticker)
#Genero el panel
panel_opciones = obtener_panel_opciones_nyse(ticker, True)



In [ ]:
panel_opciones

In [ ]:
# Tomo los TTMs y me quedo con alguno de ellos
TTMs = list(set(panel_opciones.TTM.values))
TTMs.sort()

TTM=TTMs[4]
TTM

In [ ]:
# elijo un tipo de opcion y tomo los strikes
CP = "P"
Ks = panel_opciones[(panel_opciones.TTM == TTM) & (panel_opciones.CallPut == CP)].Strike.values
Ks_indexes = panel_opciones[(panel_opciones.TTM == TTM) & (panel_opciones.CallPut == CP)].Strike.index

Ks_indexes

In [ ]:
#Tomo el ATM strike
K_atm_index = np.abs(Ks - panel_opciones.Spot.values[0]).argmin()

K_ATM = Ks[K_atm_index]
K_ATM
K_ATM_index_df = panel_opciones[(panel_opciones.TTM == TTM) & (panel_opciones.CallPut == CP)&(panel_opciones.Strike==K_ATM)].Strike.index[0]
K_ATM_index_df


In [ ]:
panel_opciones[panel_opciones.index==543]

In [ ]:
# Saco de los datos de mercado los parametros necesarios para pricear


CP = 'C'
S = panel_opciones.Spot.values[K_ATM_index_df]

K = panel_opciones.Strike.values[K_ATM_index_df]
T = (panel_opciones.TTM.values[K_ATM_index_df])/365

sigma = panel_opciones.impliedVolatility.values[K_ATM_index_df]

#sigma = panel_opciones[(panel_opciones.TTM == TTM) & (panel_opciones.CallPut == CP) & (panel_opciones.Strike == K_ATM)].impliedVolatility.values[0]
try:
    div = float(data.info['dividendYield'])
except:
    div = 0.0 
    
pasos = 2000




In [ ]:
div


In [ ]:
#tasa libre de riesgo de Libor1M - https://www.global-rates.com/en/interest-rates/libor/libor.aspx
r = 0.0543  #Libor1M**

   

In [ ]:
# Volatilidad sacada del dataframe
sigma

In [ ]:
# dividendo sacada de data.info
div


In [ ]:
#spot sacada de data.info
S

In [ ]:
# Llamo a los 2 pricers y comparo con el precio de mercado

precio_modelo_bin = opcion_americana_bin(CP, S, K, T, r, sigma, div, pasos)
print("Precio Modelo Binomial: ", precio_modelo_bin)

precio_modelo_bs = opcion_europea_bs(CP, S, K, T, r, sigma, div)
print("Precio Modelo BS: ", precio_modelo_bs)

precio_mercado_last = panel_opciones[(panel_opciones.TTM == TTM) & (panel_opciones.CallPut == CP) & (panel_opciones.Strike == K_ATM)].Last.values[0] 

precio_mercado_avg = (panel_opciones[(panel_opciones.TTM == TTM) & (panel_opciones.CallPut == CP) & (panel_opciones.Strike == K_ATM)].Bid.values[0]#
+panel_opciones[(panel_opciones.TTM == TTM) & (panel_opciones.CallPut == CP) & (panel_opciones.Strike == K_ATM)].Ask.values[0]
)/2

print("Precio Mercado Last: ", precio_mercado_last)

print("Precio Mercado Avg: ", precio_mercado_avg)



In [ ]:
#Error relativo entre valor de mercado y valor de modelo:

print("Error relativo entre modelo y mercado (en %): ", (precio_mercado_avg-precio_modelo_bs)/precio_mercado_avg*100)

## b) Saco una opcion del mercado y calculo su precio - BYMA


## Ejercicio